In [1]:
import re

from datasets import load_dataset, load_metric
import evaluate
import nltk
import nltk.data
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import (
    AdamW, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JustinTo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Setup and Preprocessing for Model Load

In [46]:
DATASET_NAME = "multi_x_science_sum"
DOC_SEP = " ||||| "
BATCH_SIZE = 1
MAX_LENGTH_ENC = 4096
MAX_LENGTH_DEC = 256

rouge = load_metric("rouge")

dataset = load_dataset(DATASET_NAME)

pat = re.compile("@cite_[0-9]+")

Found cached dataset multi_x_science_sum (C:/Users/JustinTo/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729)


  0%|          | 0/3 [00:00<?, ?it/s]

In [47]:
def preprocess_dataset(example):

    abstracts = example["abstract"].split("| Abstract: ")[-1]
    related_work = pat.sub("@cite", example["related_work"])
    ref_abstracts = filter(bool, example["ref_abstract"]["abstract"])
    output = {
        "abstracts": f"{abstracts}{DOC_SEP}{DOC_SEP.join(ref_abstracts)}",
        "related_work": related_work
    }
    return output

def preprocess_dataset_batched(example):
    abstracts = [
        abstract.split("| Abstract: ")[-1] + DOC_SEP + DOC_SEP.join([x for x in ref_abstract["abstract"] if x])
        for abstract, ref_abstract in zip(example["abstract"], example["ref_abstract"])
    ]
    related_work = [pat.sub("@cite", rw) for rw in example["related_work"]]
    output = {
        "abstracts": abstracts,
        "related_work": related_work,
    }
    return output

dataset_processed = {}
for split in dataset.keys():
    dataset_processed[split] = dataset[split].map(
        preprocess_dataset_batched,
        remove_columns=dataset[split].column_names,
        batched=True,
        batch_size=BATCH_SIZE,
    )

  0%|          | 0/30369 [00:00<?, ?ba/s]

  0%|          | 0/5093 [00:00<?, ?ba/s]

  0%|          | 0/5066 [00:00<?, ?ba/s]

In [48]:
dataset_processed

{'train': Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 30369
 }),
 'test': Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 5093
 }),
 'validation': Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 5066
 })}

## Loading the Model and Tokenizer

In [49]:
def get_tokenizer(host_tokenizer: str):
  """return the tokenizer and model for LLM training"""

  return (AutoTokenizer.from_pretrained(host_tokenizer, 
                                        gradient_checkpointing=True,use_cache=False), AutoModelForSeq2SeqLM.from_pretrained(host_tokenizer))


centrum_tokenizer, centrum_model = get_tokenizer("ratishsp/Centrum") 

loading file vocab.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\snapshots\e9e32bd7ab7f460c1786f42e6e7f4f5697ace02d\vocab.json
loading file merges.txt from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\snapshots\e9e32bd7ab7f460c1786f42e6e7f4f5697ace02d\merges.txt
loading file tokenizer.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\snapshots\e9e32bd7ab7f460c1786f42e6e7f4f5697ace02d\tokenizer.json
loading file added_tokens.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\snapshots\e9e32bd7ab7f460c1786f42e6e7f4f5697ace02d\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\snapshots\e9e32bd7ab7f460c1786f42e6e7f4f5697ace02d\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--ratishsp--Centrum\s

In [50]:
print(centrum_tokenizer, centrum_model)

PreTrainedTokenizerFast(name_or_path='ratishsp/Centrum', vocab_size=50265, model_max_len=16384, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}) LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50266, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens

In [51]:
def tokenize_dataset_batched(example):
    # Tokenizer input
    input_encoding = centrum_tokenizer(
        example["abstracts"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH_ENC,
        return_tensors="pt",
    )

    # Tokenizer output
    output_encoding = centrum_tokenizer(
        example["related_work"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH_DEC,
        return_tensors="pt",
    )

    # Modify output encoding to ignore padding in loss function
    # torch ignore -100 in loss function computation
    labels = output_encoding["input_ids"].clone()
    labels[labels == centrum_tokenizer.pad_token_id] = -100

    # Global attention with vectorized operations (optimized for GPU)
    input_ids = input_encoding["input_ids"]
    docsep_token_id = centrum_tokenizer.convert_tokens_to_ids(DOC_SEP)
    global_attention_mask = (input_ids == centrum_tokenizer.cls_token_id) | (input_ids == docsep_token_id)

    return {
        "input_ids": input_encoding["input_ids"],
        "attention_mask": input_encoding["attention_mask"],
        "global_attention_mask": global_attention_mask.float(),
        "labels": labels,
    }

centrum_tokenizer.add_tokens(DOC_SEP, special_tokens=True)
centrum_model.resize_token_embeddings(len(centrum_tokenizer))
docsep_token_id = centrum_tokenizer.convert_tokens_to_ids(DOC_SEP)

dataset_tokenized = {}
for split in dataset_processed.keys():
    dataset_tokenized[split] = (
        dataset_processed[split]
        .select(range(200))
        .map(
            tokenize_dataset_batched,
            remove_columns=dataset_processed[split].column_names,
            batched=True,
            batch_size=BATCH_SIZE,
        )
    )

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

In [52]:
dataset_tokenized

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
     num_rows: 200
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
     num_rows: 200
 }),
 'validation': Dataset({
     features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
     num_rows: 200
 })}

## Fine-tuning the Model with Trainer

In [64]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    fp16=True,
    num_train_epochs=1,
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=250,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    gradient_accumulation_steps=8
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [65]:
trainer = Seq2SeqTrainer(
    model=centrum_model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["validation"],
)

Using cuda_amp half precision backend


In [70]:
trainer.train()

***** Running training *****
  Num examples = 200
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 8
  Total optimization steps = 25
  Number of trainable parameters = 152408832


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=25, training_loss=3.207005615234375, metrics={'train_runtime': 64.5312, 'train_samples_per_second': 3.099, 'train_steps_per_second': 0.387, 'total_flos': 540040966963200.0, 'train_loss': 3.207005615234375, 'epoch': 1.0})

In [38]:
import gc

In [69]:
gc.collect()

0

In [68]:
from torchsummary import summary
summary(centrum_model)

Layer (type:depth-idx)                             Param #
├─LEDModel: 1-1                                    --
|    └─Embedding: 2-1                              38,605,056
|    └─LEDEncoder: 2-2                             --
|    |    └─Embedding: 3-1                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-2     3,145,728
|    |    └─ModuleList: 3-3                        53,157,888
|    |    └─LayerNorm: 3-4                         1,536
|    └─LEDDecoder: 2-3                             --
|    |    └─Embedding: 3-5                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-6     786,432
|    |    └─ModuleList: 3-7                        56,710,656
|    |    └─LayerNorm: 3-8                         1,536
├─Linear: 1-2                                      38,605,056
Total params: 191,013,888
Trainable params: 191,013,888
Non-trainable params: 0


Layer (type:depth-idx)                             Param #
├─LEDModel: 1-1                                    --
|    └─Embedding: 2-1                              38,605,056
|    └─LEDEncoder: 2-2                             --
|    |    └─Embedding: 3-1                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-2     3,145,728
|    |    └─ModuleList: 3-3                        53,157,888
|    |    └─LayerNorm: 3-4                         1,536
|    └─LEDDecoder: 2-3                             --
|    |    └─Embedding: 3-5                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-6     786,432
|    |    └─ModuleList: 3-7                        56,710,656
|    |    └─LayerNorm: 3-8                         1,536
├─Linear: 1-2                                      38,605,056
Total params: 191,013,888
Trainable params: 191,013,888
Non-trainable params: 0